# UPS MV Framework

### Import Libraries

In [ ]:
from AssetAllocation.datamanger import datamanger as dm
from AssetAllocation.analytics import summary
from AssetAllocation.reporting import reports as rp, plots, formatter as fmt
from ipywidgets import interact, interact_manual


### Compute inputs (return, volatility and correlation) for plan

In [ ]:
#get return
mv_inputs_dict = dm.get_mv_inputs_data('inputs_test.xlsx')
mv_inputs = summary.get_mv_inputs(mv_inputs_dict)
#get historical vol and correlation
ts_dict = dm.get_ts_data('return_data.xlsx')
ts_outputs = summary.get_ts_output(ts_dict)
pp_inputs = summary.get_pp_inputs(mv_inputs, ts_dict)

### Display plan inputs

In [ ]:
@interact
def display_pp_inputs(inputs=pp_inputs.keys()):
    if inputs == 'corr':
        return plots.draw_heatmap(pp_inputs[inputs], half=False)
    else:
        return fmt.get_percent_styler(pp_inputs[inputs])

### Create plan object that has the relevant variables (cov, policy ret, var,fsv )

In [ ]:
plan = summary.get_plan_params(pp_inputs)

pp_dict = summary.get_pp_dict(plan)
@interact
def display_pp_inputs(variable=pp_dict.keys()):
    if variable == 'Corr' or variable == 'Cov':
        return plots.draw_heatmap(pp_dict[variable], half=False)
    else:
        return fmt.get_percent_styler(pp_dict[variable])

### Define bounds

In [ ]:
bnds = dm.get_bounds(dm.OUTPUTS_FP+'bounds.xlsx')

### Define contraints to optimize for min and max return

In [ ]:
cons = ({'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-1]) - x[3] - .02})

### Define bounds

In [ ]:
# Optimizing for maximum sharpe ratio
opt_sharpe = plan.optimize(plan.min_sharpe_ratio, bnds, cons)

### Compute min return


In [ ]:
# Optimizing for minimum variance portfolio
opt_var = plan.optimize(plan.min_variance,bnds, cons)
# set the return as min_ret
min_ret = plan.portfolio_stats(opt_var['x'])[0]

### Compute max return


In [ ]:
# Optimizing for max return portfolio
opt_ret = plan.optimize(plan.min_ret,bnds, cons)
# set the return as max_ret
max_ret = np.around(plan.portfolio_stats(opt_ret['x']),4)[0]

### Get data for MV Efficient Frontier Portfolios

In [ ]:
#Getting data for efficient frontier portfolios
target_rets = np.linspace(min_ret,max_ret,100)
t_vols = []
t_weights = []

for tr in target_rets:
    
    ef_cons = ({'type': 'eq', 'fun': lambda x: plan.portfolio_stats(x)[0] - tr},
               {'type': 'ineq', 'fun': lambda x: np.sum(x[1:3]) - 0.5},
        {'type': 'eq', 'fun': lambda x: np.sum(x[0:len(plan)-1])-x[3] - .02})
    
    opt_ef = plan.optimize(plan.min_volatility, bnds, ef_cons)
    
    t_vols.append(opt_ef['fun'])
    t_weights.append(opt_ef['x'])
    
target_vols = np.array(t_vols)
target_weights = np.array(t_weights)
optimized_weights = np.transpose(target_weights)

### Display MV Efficient Frontier portfolios

In [ ]:
#Dataframe containing efficient frontier portfolios (return, vol, sharpe and weights)
ports_df = dm.get_ports_df(target_rets, target_vols, target_weights, plan.symbols, raw=True)
fmt.get_port_styler(ports_df)

### Display MV Asset Allocation

In [ ]:
#Asset Allocation Plot
aa_fig = plots.get_aa_fig(100*np.around(ports_df,6))
aa_fig.show()

### Display MV Efficient Frontier

In [ ]:
#Plotly version of the Efficient Frontier plot
ef_fig = plots.get_ef_fig(ports_df)
ef_fig.show()

### Export data to excel

In [ ]:
#Export Efficient Frontier portfoio data to excel
rp.get_ef_portfolios_report('ef_portfolios', ports_df,pp_inputs)